In [ ]:
# ! pip3 uninstall ffmpeg-python

In [95]:
# Video: cumulative frames

import ffmpeg
import imageio
import numpy as np
import os
from PIL import Image, ImageFont, ImageDraw

WIDTH = 2000
HEIGHT= 2000

ROOT_PREFIX = "data_venus/arecibo_radar/pds-geosciences.wustl.edu/venus/arcb_nrao-v-rtls_gbt-3-delaydoppler-v1/vrm_90xx/"
IMAGE_PREFIX = ROOT_PREFIX + "OCP_GLOBAL_TRIAGE/"

font = ImageFont.truetype(r"/mnt/c/Windows/Fonts/arial.ttf", 64)

image_filenames = os.listdir(IMAGE_PREFIX)
image_filenames.sort()

ffmpeg_process = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='gray8', s='{}x{}'.format(WIDTH, HEIGHT))
    .output(IMAGE_PREFIX + 'video_cumulative.mp4', pix_fmt='gray8', r=10)
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

buffer = np.zeros((HEIGHT, WIDTH), np.uint8)
counter = 0
for image_filename in image_filenames:
    if not image_filename.endswith('.png'): continue
    image = imageio.v3.imread(IMAGE_PREFIX + image_filename)
    image = image[:, 2500:10500]  # crop
    #image = image[::4, ::4]  # Direct downsample

    # Downscale with 4x4 average, avoiding averages with 0
    image = image.reshape(HEIGHT, 4, WIDTH, 4) 
    image = image.mean((1, 3), where=image!=0)
    image = image.astype(np.uint8)

    # 50% blended new frame
    buffer = buffer.astype(np.float32)
    buffer = np.where(image != 0, (image + buffer) / 2, buffer)
    buffer = buffer.astype(np.uint8)

    im = Image.fromarray(buffer)
    draw = ImageDraw.Draw(im)
    draw.text((5, 5), image_filename[10:-4], "White", font=font)

    #ffmpeg_process.stdin.write(image.tobytes())
    #ffmpeg_process.stdin.write(buffer.tobytes())
    ffmpeg_process.stdin.write(im.tobytes())

    # Full new frame
    buffer = np.where(image != 0, image, buffer)
    buffer = buffer.astype(np.uint8)

    im = Image.fromarray(buffer)
    draw = ImageDraw.Draw(im)
    draw.text((5, 5), image_filename[10:-4], "White", font=font)

    #ffmpeg_process.stdin.write(image.tobytes())
    #ffmpeg_process.stdin.write(buffer.tobytes())
    ffmpeg_process.stdin.write(im.tobytes())

    counter += 1
    #if counter > 10: break

ffmpeg_process.stdin.close()
ffmpeg_process.wait()

/home/than/.local/lib/python3.10/site-packages/PIL/Image.py:3074: DecompressionBombWarning: Image size (128000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-libru

0

In [100]:
# Video: Cumulative average

import ffmpeg
import imageio
import numpy as np
import os
from PIL import Image, ImageFont, ImageDraw

WIDTH = 2000
HEIGHT= 2000

ROOT_PREFIX = "data_venus/arecibo_radar/pds-geosciences.wustl.edu/venus/arcb_nrao-v-rtls_gbt-3-delaydoppler-v1/vrm_90xx/"
#IMAGE_PREFIX = ROOT_PREFIX + "GLOBAL_TRIAGE/"
IMAGE_PREFIX = ROOT_PREFIX + "OCP_GLOBAL_TRIAGE/"

font = ImageFont.truetype(r"/mnt/c/Windows/Fonts/arial.ttf", 64)

image_filenames = os.listdir(IMAGE_PREFIX)
image_filenames.sort()

ffmpeg_process = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='gray8', s='{}x{}'.format(WIDTH, HEIGHT))
    .output(IMAGE_PREFIX + 'video_cumulative_average.mp4', pix_fmt='gray8', r=6)
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

buffer = np.zeros((8000, 8000), np.float32)
counts = np.zeros((8000, 8000), np.int16)
counter = 0
for image_filename in image_filenames:
    if not image_filename.endswith('.png'): continue
    image = imageio.v3.imread(IMAGE_PREFIX + image_filename)
    image = image[:, 2500:10500]  # crop
    #image = image[::4, ::4]  # Direct downsample

    buffer += image
    np.add(counts, 1, out=counts, where=image!=0)

    # Downscale with 4x4 average, avoiding averages with 0
    image = buffer.copy()
    np.divide(image, counts, out=image, where=counts>0) 
    image = image.reshape(HEIGHT, 4, WIDTH, 4) 
    image = image.mean((1, 3), where=image!=0)
    image = image.astype(np.uint8)

    im = Image.fromarray(image)
    draw = ImageDraw.Draw(im)
    draw.text((5, 5), image_filename[10:-4], "White", font=font)
    ffmpeg_process.stdin.write(im.tobytes())

    counter += 1
    #if counter > 10: break

ffmpeg_process.stdin.close()
ffmpeg_process.wait()

/home/than/.local/lib/python3.10/site-packages/PIL/Image.py:3074: DecompressionBombWarning: Image size (128000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-libru